In [ ]:
import tensorflow as tf
from tensorflow import keras
from tqdm.keras import TqdmCallback
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def poiseuille(R=1,dpdx=-0.01,nu=0.01,Nr=100,figure=True):
  
    r = np.linspace(-R,R,Nr)
    u = -1/4/nu *dpdx*(R**2-r**2)
    if figure:
        fig, ax = plt.subplots(figsize=(10,4))
        ax.scatter(r,u)
        ax.set_ylabel("u [m/s]")
        ax.set_xlabel("r [m]")
    umax = -1/4/nu*dpdx*R*R
    Re = umax*2*R/nu
    return Re,r,u

In [ ]:
Re, r, u = poiseuille(R=1,dpdx=-0.01,nu=0.01,Nr=11,figure=True)

In [ ]:
# making the samples
rey   = []
perf  = []
for Reynolds in np.linspace(-0.99,0.99,10):
    re, r, u = poiseuille(R=Reynolds,dpdx=-0.01,nu=0.01,Nr=11,figure=False)
    rey.append([re])
    perf.append(u)
# lists to array and scale
rn = (np.array(rey))
reynolds = rn/rn.max()
pro  = np.array(perf)
profile = pro/pro.max()

In [ ]:
reynolds

In [ ]:
profile

In [ ]:
# creating layers
inputs = tf.keras.Input(shape=1, name='input')
hidden = tf.keras.layers.Dense(16, activation='relu', name='hidden_1')(inputs)
output = tf.keras.layers.Dense(11, name='output')(hidden)
# creating the model
model  = keras.Model(inputs, output, name='model')
# printing
model.summary()

In [ ]:
# compiling the model
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.1)
loss      = tf.keras.losses.MeanSquaredError()
model.compile(optimizer, loss, metrics=['mae'])
model.fit(reynolds, profile, epochs=15, verbose=0, callbacks=[TqdmCallback(verbose=1)])